In [1]:
import json
import os
import httpx
from PIL import Image # Import Pillow for image processing
import numpy as np # Import numpy for array manipulation

In [2]:
# Ensure you have these installed:
# pip install open-inference-openapi httpx Pillow numpy

from open_inference.openapi.client import OpenInferenceClient, InferenceRequest

#If running from workbench use /tmp/jwt. Otherwise provide your CDP_TOKEN
#Make sure to replace "/tmp/jwt" with the actual path to your JWT file if different
try:
    API_KEY = json.load(open("/tmp/jwt"))["access_token"]
except FileNotFoundError:
    print("Warning: /tmp/jwt not found. Please ensure you have a valid CDP_TOKEN or JWT file.")
    # You might want to prompt the user for API_KEY here or exit


BASE_URL = ''
MODEL_NAME = ''
headers = {
	'Authorization': 'Bearer ' + API_KEY,
	'Content-Type': 'application/json'
}

httpx_client = httpx.Client(headers=headers)
client = OpenInferenceClient(base_url=BASE_URL, httpx_client=httpx_client)

# Check that the server is live, and it has the model loaded
try:
    client.check_server_readiness()
    print("Server is ready.")
    metadata = client.read_model_metadata(MODEL_NAME)
    metadata_str = json.dumps(json.loads(metadata.json()), indent=2)
    print("\nModel Metadata:")
    print(metadata_str)

    # --- Extract input details from metadata ---
    # Assuming the model has at least one input and it's an image
    model_inputs_meta = json.loads(metadata.json()).get("inputs", [])
    if not model_inputs_meta:
        raise ValueError("Model metadata does not contain input information.")

    # For simplicity, we'll assume the first input in the metadata is the one we need.
    # In a real application, you might need to iterate or check names if there are multiple inputs.
    input_name = model_inputs_meta[0]["name"]
    input_shape = model_inputs_meta[0]["shape"]
    input_datatype = model_inputs_meta[0]["datatype"]

    print(f"\nDetected Input Name: {input_name}")
    print(f"Detected Input Shape: {input_shape}")
    print(f"Detected Input Datatype: {input_datatype}")

    # --- Image Loading and Preprocessing ---
    # Define the path to your local JPEG image
    # IMPORTANT: Replace 'path/to/your/image.jpeg' with the actual path to your image file
    IMAGE_PATH = "/home/cdsw/person1946_bacteria_4874.jpeg" # e.g., 'patient_xray.jpeg'

    # Determine the target image size from the model's input shape
    # The model expects [-1, 1, 224, 224] -> [BATCH, CHANNEL, HEIGHT, WIDTH]
    # Pillow's resize expects (width, height)
    if len(input_shape) >= 3: # Ensure there are enough dimensions for height and width
        target_height = input_shape[-1] # Last dimension is width
        target_width = input_shape[-2] # Second to last dimension is height
        target_size = (target_height, target_width) # Pillow expects (width, height)
    else:
        raise ValueError(f"Unexpected input shape for image: {input_shape}. Cannot determine resize dimensions.")

    print(f"Target image size for model: {target_size}")

    try:
        img = Image.open(IMAGE_PATH)
        img = img.resize(target_size) # Resize image to model's expected dimensions (224x224)
        
        # Convert to grayscale ('L' for Luminance) as the model expects 1 channel
        if img.mode != 'L':
            img = img.convert('L')

        # Convert image to a numpy array
        img_array = np.asarray(img) # Shape will be (HEIGHT, WIDTH) for grayscale

        # Normalize pixel values if the model expects FP32 in a certain range (e.g., 0-1)
        # This is a common step for deep learning models. Adjust as per your model's training.
        if input_datatype == "FP32":
            img_array = img_array / 255.0 # Normalize to 0-1 range for FP32

        # Add batch dimension and channel dimension, then transpose to [BATCH, CHANNEL, HEIGHT, WIDTH]
        # Current img_array shape for grayscale is (HEIGHT, WIDTH)
        # Add channel dimension (1) at axis 0: (1, HEIGHT, WIDTH)
        # Add batch dimension (1) at axis 0: (1, 1, HEIGHT, WIDTH)
        img_array = img_array[np.newaxis, np.newaxis, :, :]

        # Verify the final shape before flattening
        print(f"Preprocessed image array shape: {img_array.shape}")

        # Flatten the numpy array to a 1D list
        image_data_flat = img_array.flatten().tolist()
        print(f"Image data flattened. Total elements: {len(image_data_flat)}")

    except FileNotFoundError:
        print(f"Error: Image file not found at {IMAGE_PATH}. Please check the path.")
        exit()
    except Exception as e:
        print(f"Error processing image: {e}")
        exit()

    # Make an inference request
    print("\nMaking inference request...")
    pred = client.model_infer(
        MODEL_NAME,
        request=InferenceRequest(
            inputs=[
                {
                    "name": input_name,
                    "shape": list(img_array.shape), # Use the actual shape of the prepared numpy array (e.g., [1, 1, 224, 224])
                    "datatype": input_datatype,
                    "data": image_data_flat
                }
            ]
        ),
    )

    json_resp_str = json.dumps(json.loads(pred.json()), indent=2)
    print("\nInference Response:")
    print(json_resp_str)

except httpx.HTTPStatusError as e:
    print(f"HTTP Error: {e.response.status_code} - {e.response.text}")
    print("Please check your BASE_URL, MODEL_NAME, and API_KEY.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Server is ready.

Model Metadata:
{
  "name": "rtcn-dmnq-p6oh-4e8f",
  "versions": [
    "1"
  ],
  "platform": "onnxruntime_onnx",
  "inputs": [
    {
      "name": "input",
      "datatype": "FP32",
      "shape": [
        -1,
        1,
        224,
        224
      ]
    }
  ],
  "outputs": [
    {
      "name": "output",
      "datatype": "FP32",
      "shape": [
        -1,
        2
      ]
    }
  ]
}

Detected Input Name: input
Detected Input Shape: [-1, 1, 224, 224]
Detected Input Datatype: FP32
Target image size for model: (224, 224)
Preprocessed image array shape: (1, 1, 224, 224)
Image data flattened. Total elements: 50176

Making inference request...

Inference Response:
{
  "model_name": "rtcn-dmnq-p6oh-4e8f",
  "model_version": "1",
  "outputs": [
    {
      "name": "output",
      "shape": [
        1,
        2
      ],
      "datatype": "FP32",
      "data": [
        -8.839442253112793,
        10.951461791992188
      ]
    }
  ]
}


In [16]:
# convert to an index
output = json.loads(json_resp_str)["outputs"][0]['data']

# Convert logits to numpy array for softmax
logits_np = np.array(output)

# Apply softmax function for numerical stability
exp_logits = np.exp(logits_np - np.max(logits_np))
probabilities = np.round(exp_logits / np.sum(exp_logits),3)

In [17]:
max_index = np.argmax(probabilities)
max_value = np.max(probabilities)

if max_index == 1:
    print(f'Diagnosis - Pneumonia : Probability {max_value}')
else:
    print(f'Diagnosis - No Pneumonia : Probability {max_value}')

Diagnosis - Pneumonia : Probability 1.0
